In [ ]:
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
from geodatasets import get_path
from shapely import geometry

from dwca.read import DwCAReader

import xarray as xr

from os.path import isfile

# Wolf data set

In [ ]:
useful_columns = ["id", "eventDate", "decimalLatitude", "decimalLongitude", "coordinateUncertaintyInMeters", 
                  "scientificName", "level0Gid", "level0Name", "level1Gid", "level1Name", "level2Gid", "level2Name", 
                  "level3Gid", "level3Name", "geodeticDatum"]

In [ ]:
with DwCAReader("./datasets/0054964-231002084531237.zip") as dwca:
    df = dwca.pd_read('occurrence.txt', parse_dates=True)[useful_columns] #load selected columns

    # Convert to GeoDataFrame:
    assert (df["geodeticDatum"][0] == df["geodeticDatum"]).all(), "Ovservations do not share coordinate reference system!"
    crs = df["geodeticDatum"][0]
    
    gdf = gpd.GeoDataFrame(df, crs=crs, geometry=gpd.points_from_xy(df["decimalLongitude"], df["decimalLatitude"]))
    gdf.drop(gdf.loc[gdf["decimalLongitude"] < 0.].index, inplace=True) #get rid of outlier

In [ ]:
world = gpd.read_file(get_path("naturalearth.land"))

# We restrict to South America.
ax = world.clip([-11, 45, 24, 60]).plot(color="white", edgecolor="black")

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color="red")

plt.show()

# REA6 mesh for Germany

In [ ]:
# Load latitudes and longitudes:
with xr.open_dataset("./datasets/bio1.nc") as ds:
    lats, lons = ds["latitude"], ds["longitude"]

if isfile("./datasets/grid_de.shp"):
   grid_de = gpd.read_file("./datasets/grid_de.shp")
else: 
    # Computing the full mesh takes a bit of time, give a nice representation of the progress:
    import sys, time
    
    def progressbar(it, prefix="", size=60, out=sys.stdout): # Python3.6+
        count = len(it)
        start = time.time()
        def show(j):
            x = int(size*j/count)
            remaining = ((time.time() - start) / j) * (count - j)
            
            mins, sec = divmod(remaining, 60)
            time_str = f"{int(mins):02}:{sec:05.2f}"
            
            print(f"{prefix}[{u'█'*x}{('.'*(size-x))}] {j}/{count} Est wait {time_str}", end='\r', file=out, flush=True)
            
        for i, item in enumerate(it):
            yield item
            show(i+1)
        print("\n", flush=True, file=out)

    # Create polygons from lat/lon:
    polygon_grid = []
    n_x, n_y = lats.shape
    for i in progressbar(range(n_x - 1), "Computing: ", 40):
        for j in range(n_y - 1):
            polygon_grid.append(geometry.Polygon([(lons[i,j], lats[i,j]), (lons[i+1,j], lats[i+1,j]), 
                                                  (lons[i+1,j+1], lats[i+1,j+1]), (lons[i,j+1], lats[i,j+1])]))

    # Convert to gdf:
    grid = gpd.GeoDataFrame(polygon_grid, columns=['geometry']).set_crs(crs)
    grid.to_file("./datasets/grid_full.shp")

    # Cut out Germany:
    de = world[world["name"] == "Germany"]
    grid_de = gpd.tools.sjoin(mesh, de)
    grid_de.to_file("./datasets/grid_de.shp")

# Compute occurences per grid point

Currently this only gives the total number of sightings. For a year by year approach one would probably filter the wolf data year by year and do the below steps individually.

In [ ]:
# Spatially join grid and observations:
merged = gpd.sjoin(gdf, grid_de)[["id", "eventDate", "index_right", "geometry"]]
merged["nOccurences"] = 1

# Count occurences:
dissolved = merged.dissolve(by="index_right", aggfunc="count") #counts how many points fall into each polygon
grid_de.loc[dissolved.index, "nOccurences"] = dissolved["nOccurences"].values #store data back into whole grid
occ_gridded = grid_de[["nOccurences", "geometry"]] #shrunk version

In [ ]:
ax = occ_gridded.plot(column="nOccurences", figsize=(12, 8), cmap='viridis', edgecolor="grey", legend=True)
plt.autoscale(False)
world.to_crs(grid_de.crs).plot(ax=ax, color='none', edgecolor='black')

plt.show()

# Compare to bioclimatic variables

In [ ]:
for i in range(1,20):
    _ = xr.open_dataset(f"./datasets/bio{i}.nc")
    _ = _[list(_.keys())[0]].isel(year=10)

    fig, axs = plt.subplots()
    
    axs.contourf(lons, lats, _) #bioclimatic variable
    occ_gridded.plot(column="nOccurences", cmap='viridis', edgecolor="grey", ax=axs) #occurences
    
    world.to_crs(crs).plot(ax=axs, color='none', edgecolor='black') #sea border
    
    axs.set(title=f"BIO{i}")
    axs.set_xlim(4, 18)
    axs.set_ylim(47, 55)
    plt.show()